# <font color='#31394d'>Practical Exercise: Linear Regression</font>

In this notebook, we are going to train a simple linear regression model using the [`scikit-learn`](https://scikit-learn.org) library. Recall that linear regression is a *supervised learning* technique that is suitable for a *continuous/numerical* outcome variable.  

We begin by importing modules for data wrangling:

<!-- 
Even though its name is scikit-learn, it is imported as `sklearn`. It has many submodules.
For example, the `datasets` submodule has a group of simple datasets that can be used to evaluate models without having to use external files.

The Boston Housing dataset is available as a scikitlearn dataset.-->

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## <font color='#31394d'> Get and Explore the Data </font>

We'll be using is the [Boston Housing](https://www.kaggle.com/c/boston-housing) dataset from Kaggle. This dataset consists of information about houses in the Boston area. Our goal is to **predict the typical price of a house**.

We import the data from the ``sklearn`` module as follows:

In [ ]:
from sklearn import datasets
boston = datasets.load_boston()

`sklearn` datasets behave like a dictionary. Let's see what this dictionary contains:

In [ ]:
boston.keys()

The `DESCR` key includes a description of the dataset:

In [ ]:
print(boston["DESCR"])

The `target` key holds the target/outcome variable; in this case, the median house value in thousands of dollars.

In [ ]:
boston["target"]

The names of the features/independent variables are stored under the `feature_names` key:

In [ ]:
boston["feature_names"]

Finally, the values of the features are stored under the `data` key:

In [ ]:
boston["data"]

Let's put the Boston data into a pandas dataframe:

In [ ]:
df = pd.DataFrame(boston["data"], columns=boston["feature_names"])

df["PRICE"] = boston["target"]

df.head()

## <font color='#31394d'> Train/Test Split </font>

Before we create the regression model, we need to split the data into training and test subsets. This way, we train on one portion of the data (the "training set") and measure model performance on the other portion (the "test" set). Usually, the training set is larger than the test set.

<!--One way to avoid method overfitting and making sure that the model doesn't memorize the dataset is to do a train test split.-->
![title](media/train_test_split.png) 


![title](media/train_test.png)

We can use the `train_test_split` function from the `sklearn` module to easily split the dataset into training and test subsets. 

`train_test_split` works both with numpy arrays and pandas dataframes. If we pass it with a numpy array, it will return 4 different arrays: an X,y pair for training and another X,y pair for the test dataset. If we pass it a pandas dataframe, it will split the dataframe into two (training and test dataframes). 

We use the argument `test_size` to define the % size of the test dataset.

The full dataset is divided row-wise into training and test sets *at random*. This means that if we run the `train_test_split` twice, we will get different datasets. In order to make sure that we get the same splits again and again, we can fix the *random seed*, that is, the number that numpy uses to start its random number generation (used to calculate the splits). We can use the argument `random_state` to set the random seed for `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(df, test_size=0.2, random_state=12345)


print('Training set has', train.shape[0], 'rows')
print('Test set has', test.shape[0], 'rows')

## <font color='#31394d'> Model Fitting </font>

The algorithms for linear regression are in the `linear_model` submodule of `sklearn`. Let's import the `LinearRegression` class and create (instantiate) an *estimator* object. Note that this is the standard procedure for any machine learning algo available in `sklearn`. 

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

We train the model (i.e. estimate its parameters) on the training data with the `fit` method. The `fit` method follows the same structure for pretty much every model in scikit-learn. We pass as arguments the features `X` and outcome variable `y`. The find out more, recall that you can type `?model.fit`.

Let's fit a simple linear model with house price as the outcome and number of rooms as the feature:

In [ ]:
# train['RM']  is a pandas series
# train[['RM']] is a pandas dataframe
# NB: the fir method expects X to be a dataframe, not a series 

model.fit(X=train[["RM"]], y=train["PRICE"])

Now that the linear regression model is trained (fitted), it has two additional attributes: `coef_` is an array containing the estimated coefficients/slopes for each feature, and `intercept_` contains the estimated intercept term.

In [ ]:
model.intercept_ # theta0 estimate

In [ ]:
model.coef_ # theta1 estimate

In [ ]:
print('The estimated regression function is:\n\nave price =', np.round(model.intercept_,2), '+', np.round(model.coef_[0],2), '* rooms')

🚀 <font color='#d9c4b1'> Exercise: </font> Does the estimated slope make sense to you?
    
🚀 <font color='#d9c4b1'> Exercise: </font> Does the estimated intercept make sense to you? Hint: How would you interpret it?

In practice, we would have *first* produced some plots and done some exploratory analysis before assuming a linear relationship between price and the number of rooms. 

Let's see what how well our model describes the training data:

In [ ]:
sns.regplot(x=train[['RM']], y=train['PRICE'])

## <font color='#31394d'> Model Evaluation </font>

Once the model is trained, we can use the estimator object's `predict` method to get predicted house prices for the test set:

In [ ]:
y_hat = model.predict(test[['RM']])
y_hat

Let's compute some performance metrics. We can either do this by hand...

In [ ]:
np.mean((test['PRICE'] - y_hat)**2) # mean squared error 

...Or we can use the functions available scikit-learn's `metrics` submodule. These functions take the actual values and predicted values of the outcome variable as arguments:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mean_squared_error(y_true=test['PRICE'], y_pred=y_hat)

In [ ]:
mean_absolute_error(y_true=test['PRICE'], y_pred=y_hat)

The magnitudes of the MSE and MAE are dependent on how the outcome variable is measured. They are therefore not comparable across datasets, but are useful for model and feature selection on a given dataset.

Let's visualize the actual price against the predicted price. If the model was perfect, the predicted values would equal the observed values, and all the points would lie on the diagonal line through the origin:

In [ ]:
sns.relplot(x=test['PRICE'], y=y_hat, kind="scatter")

🚀 <font color='#d9c4b1'> Exercise: </font> Regress price against age and determine if this model is better than the one that uses the number of rooms as a feature. 

🚀 <font color='#d9c4b1'> Advanced Exercise: </font> Write a function that, for any single feature, regresses price against that feature on the training set, and computes the evaluation metrics on the test set. Then use your function to determine which feature is best at predicting price. 

## <font color='#31394d'> Cross Validation </font>

Cross validation is an alternative approach to evaluate out-of-sample model performance. To do cross validation, we simply split the data into *K* folds, and for each fold, we train the model on the data from the *K*-1 remaining folds and evaluate on the one that was not included in the training set. That way, we get out-of-sample predictions and errors for every data point, so we don't rely on a single test set. 

For example, a 5 fold cross validation would look like this:

![title](media/cross_validation.png)

The `cross_val_score` function in `scikit-learn` computes your choice of evaluation metric for each fold. To use this function, we first need to see what "scoring methods" are available:

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

Looks like it defines the evaluation metrics such that "bigger is better". So, if we want to use MSE, for example, we need to choose "neg_mean_squared_error" (the negative MSE)...

In [ ]:
from sklearn.model_selection import cross_val_score

model = LinearRegression()

cv_scores = cross_val_score(estimator=model, X=df[['RM']], y=df['PRICE'], scoring="neg_mean_squared_error", cv=5)
cv_scores

Note that running this function again will produce different results since the data are split into folds randomly each time the function is called. 

The cross-validated MSE for the simple regression model of price against room is therefore:

In [ ]:
-cv_scores.mean()

🚀 <font color='#d9c4b1'> Exercise: </font> Compute the CV score for price regressed against another feature in the dataset. Which model is best?

If we want to get more information about each split, we can use the `cross_validate` function instead. It also accepts multiple scoring functions/evaluation metrics. Think of `cross_val_score` as the simplified version of `cross_validate`...

In [ ]:
from sklearn.model_selection import cross_validate
scoring_functions = {"negMSE": "neg_mean_squared_error", "negMAE": "neg_mean_absolute_error"}
cv_info = cross_validate(estimator=model, X=df[['RM']], y=df['PRICE'], scoring=scoring_functions, cv=10, return_train_score=True)
cv_df = pd.DataFrame(cv_info)
cv_df

We get results for each one of the folds:
- fit time = how long it took to train the model
- score time = how long it took to make predictions and compute the score
- test and train scores are given for each one of the scoring functions